In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd
from coord_convert.transform import wgs2gcj, wgs2bd, gcj2wgs, gcj2bd, bd2wgs, bd2gcj
import matplotlib.pyplot as plt
import os
import transbigdata as tbd
import sys
sys.path.append("../GTWR_ana/")
from datetime import datetime, timezone, timedelta
from matplotlib import ticker
import DataMerge
import DataClearn
import regre_gtwr
import data_st_vis
import data_norn
import spatial_autocorr
import copy
import time, datetime
import importlib
importlib.reload(DataMerge)
importlib.reload(DataClearn)
importlib.reload(regre_gtwr)
importlib.reload(data_st_vis)
importlib.reload(data_norn)
importlib.reload(spatial_autocorr)

<module 'spatial_autocorr' from '../GTWR_ana\\spatial_autocorr.py'>

# 1.Integration of independent and dependent variables
## 1.1 Aggregation of Dependent and independent variables

In [7]:
# Compilation of datasets with casualties
x_file_list = ["../../data/各时间片各栅格交通状态数据/avg_traffic_freq.csv"]
DataMerge.data_agg.merge(y_file="../../data/各时间片各栅格事故频率数据/inj_freq_ts_data.csv",
               x_file_list=x_file_list,
               grid_data_file="../../data/ResearchAreaGrid/grid/research_grid.shp",
               merge_data_save_file="../../data/merge_data/inj_data/inj_merge_data.shp")

In [9]:
# Compilation of Datasets for Property Damage Only
x_file_list = ["../../data/各时间片各栅格交通状态数据/avg_traffic_freq.csv"]
DataMerge.data_agg.merge(y_file="../../data/各时间片各栅格事故频率数据/pdo_freq_ts_data.csv",
               x_file_list=x_file_list,
               grid_data_file="../../data/ResearchAreaGrid/grid/research_grid.shp",
               merge_data_save_file="../../data/merge_data/pdo_data/pdo_merge_data.shp")

In [5]:
# Organize the dataset with casualties and generate the final file
x_file_list = ["../../code/OSM道路设施/osm数据道路设施情况.csv","../../code/POI建成环境/建成环境.csv"]
DataMerge.data_agg.merge_staic_variable(x_file_list=x_file_list,
                   grid_data_file="../../data/merge_data/inj_data/inj_merge_data.shp",
                   merge_data_save_file="../../data/merge_data/inj_data_all/inj_merge_data.shp")
df = gpd.read_file("../../data/merge_data/inj_data_all/inj_merge_data.shp")
df.head(1)

../GTWR_ana\DataMerge.py:55: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(merge_data_save_file)


,LONCOL,LATCOL,ts,acci_freq,avg_freq,in_degree,out_degree,node_betwe,pagerank,street_den,intersecti,circuity_a,edge_densi,orientatio,bike_stree,cr1,cr2,cr3,cr4,cr5,cr6,normalized,geometry
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.389523,0.0,0.610477,0.386853,"POLYGON ((120.22035 30.13370, 120.22035 30.14269, 120.23076 30.14269, 120.23076 30.13370, 120.22035 30.13370))"


In [12]:
# Compile the dataset for property damage only and generate the final file
x_file_list = ["../../code/OSM道路设施/osm数据道路设施情况.csv","../../code/POI建成环境/建成环境.csv"]
DataMerge.data_agg.merge_staic_variable(x_file_list=x_file_list,
                   grid_data_file="../../data/merge_data/pdo_data/pdo_merge_data.shp",
                   merge_data_save_file="../../data/merge_data/pdo_data_all/pdo_merge_data.shp")
df = gpd.read_file("../../data/merge_data/pdo_data_all/pdo_merge_data.shp")
df.head(1)

../GTWR_ana\DataMerge.py:55: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(merge_data_save_file)


,LONCOL,LATCOL,ts,acci_freq,avg_freq,in_degree,out_degree,node_betwe,pagerank,street_den,intersecti,circuity_a,edge_densi,orientatio,bike_stree,cr1,cr2,cr3,cr4,cr5,cr6,normalized,geometry
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.389523,0.0,0.610477,0.386853,"POLYGON ((120.22035 30.13370, 120.22035 30.14269, 120.23076 30.14269, 120.23076 30.13370, 120.22035 30.13370))"


## 1.2 Data cleansing and preprocessing

In [15]:
# Data restoration for grids with traffic accidents not zero but traffic volume zero (only property damage)
DataClearn.clearn.clearn_traffic_state(merge_data="../../data/merge_data/pdo_data_all/pdo_merge_data.shp",
                            data_save_file="../../data/merge_data/pdo_data_clearn/pdo_merge_data.shp")

In [16]:
# Data repair for grids with traffic accidents not zero but traffic volume zero (with casualties)
DataClearn.clearn.clearn_traffic_state(merge_data="../../data/merge_data/inj_data_all/inj_merge_data.shp",
                            data_save_file="../../data/merge_data/inj_data_clearn/inj_merge_data.shp")

In [17]:
# Renaming only property loss data
DataClearn.clearn.field_rename(merge_data="../../data/merge_data/pdo_data_clearn/pdo_merge_data.shp",
                        data_save_file="../../data/merge_data/pdo_data_rename/pdo_merge_data.shp")

In [18]:
# Renaming of personnel injury data exists
DataClearn.clearn.field_rename(merge_data="../../data/merge_data/inj_data_clearn/inj_merge_data.shp",
                        data_save_file="../../data/merge_data/inj_data_rename/inj_merge_data.shp")

In [19]:
# Shrinking processing of property loss data only
DataClearn.clearn.winsor_data(merge_data="../../data/merge_data/pdo_data_rename/pdo_merge_data.shp" ,
                       explain_var_list=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12',
                                         'x13','x14','x15','x16','x17','x18'] ,
                       data_save_file="../../data/merge_data/pdo_data_rename/pdo_merge_data.shp")

In [20]:
# Shrinking processing of personnel injury data
DataClearn.clearn.winsor_data(merge_data="../../data/merge_data/inj_data_rename/inj_merge_data.shp" ,
                       explain_var_list=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12',
                                         'x13','x14','x15','x16','x17','x18'] ,
                       data_save_file="../../data/merge_data/inj_data_rename/inj_merge_data.shp")

In [21]:
# Standardize only property loss data
data_norn.norm.start(data_file="../../data/merge_data/pdo_data_rename/pdo_merge_data.shp",
               output_file="../../data/merge_data/pdo_data_norm/pdo_merge_data.shp")

In [22]:
# Standardization of casualties data
data_norn.norm.start(data_file="../../data/merge_data/inj_data_rename/inj_merge_data.shp",
               output_file="../../data/merge_data/inj_data_norm/inj_merge_data.shp")

# 2.Fitting
## 2.1 Collinearity judgment and spatial autocorrelation judgment

In [23]:
# 共线性判断
regre_gtwr.regression.cal_VIF(data_file="../../data/merge_data/pdo_data_norm/pdo_merge_data.shp",
                       explain_var_list=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12',
                                         'x13','x14','x15','x16','x17','x18'])

const     1.000000
x1        1.174995
x2       40.114016
x3       37.398221
x4        1.403615
x5        1.599282
x6       56.516765
x7        4.971021
x8        1.394214
x9       29.807749
x10       2.407424
x11      34.566168
x12       3.679095
x13       2.212647
x14       2.003127
x15       1.790075
x16       2.017787
x17       2.336235
x18       4.791538
dtype: float64


In [24]:
# 共线性判断
regre_gtwr.regression.cal_VIF(data_file="../../data/merge_data/inj_data_norm/inj_merge_data.shp",
                       explain_var_list=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12',
                                         'x13','x14','x15','x16','x17','x18'])

const     1.000000
x1        1.271896
x2       40.493604
x3       37.674204
x4        1.406573
x5        1.619934
x6       56.454735
x7        4.980235
x8        1.395671
x9       29.779962
x10       2.407791
x11      34.550355
x12       3.669176
x13       2.210987
x14       2.018226
x15       1.785456
x16       1.992803
x17       2.316060
x18       4.802528
dtype: float64


In [5]:
# 空间自相关性判断
for tmp_var in ['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18']:
    spatial_autocorr.autocorr.start(data_file="../../data/merge_data/pdo_data_norm/pdo_merge_data.shp",
                   test_var=tmp_var)

正在检验的变量为: x1
Moran's I 值为： 0.1314503491911806
随机分布假设下Z检验值为： 60.5880296442641
随机分布假设下Z检验的P值为： 0.0
正态分布假设下Z检验值为： 60.59944487854433
正态分布假设下Z检验的P值为： 0.0
正在检验的变量为: x4
Moran's I 值为： 0.1807774971258599
随机分布假设下Z检验值为： 83.23644381179672
随机分布假设下Z检验的P值为： 0.0
正态分布假设下Z检验值为： 83.26060218810345
正态分布假设下Z检验的P值为： 0.0
正在检验的变量为: x5
Moran's I 值为： 0.07540903399366293
随机分布假设下Z检验值为： 34.84710392112268
随机分布假设下Z检验的P值为： 0.0
正态分布假设下Z检验值为： 34.85376298330766
正态分布假设下Z检验的P值为： 0.0
正在检验的变量为: x7
Moran's I 值为： 0.2710668642060623
随机分布假设下Z检验值为： 124.70758555489827
随机分布假设下Z检验的P值为： 0.0
正态分布假设下Z检验值为： 124.7400234896241
正态分布假设下Z检验的P值为： 0.0
正在检验的变量为: x8
Moran's I 值为： 0.23057100353419108
随机分布假设下Z检验值为： 106.11338212052183
随机分布假设下Z检验的P值为： 0.0
正态分布假设下Z检验值为： 106.13600709105813
正态分布假设下Z检验的P值为： 0.0
正在检验的变量为: x10
Moran's I 值为： 0.28093594388963683
随机分布假设下Z检验值为： 129.24210442632375
随机分布假设下Z检验的P值为： 0.0
正态分布假设下Z检验值为： 129.27393182558168
正态分布假设下Z检验的P值为： 0.0
正在检验的变量为: x12
Moran's I 值为： 0.18834545294571786
随机分布假设下Z检验值为： 86.71627272869205
随机分布假设下Z检验的P

## 2.2 GTWR

In [25]:
## PDO
regre_gtwr.regression.gtwr_start(data_file="../../data/merge_data/pdo_data_norm/pdo_merge_data.shp",
                          explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18'],
                          result_data_save_file="../../data/GTWR_result/pdo_betas/betas.shp")

bw:  5.1 , tau:  9.7 , score:  4793.812630895792
bw:  3.2 , tau:  3.0 , score:  4663.579861865374
bw:  1.9 , tau:  0.6 , score:  4483.5440590395265
bw:  1.2 , tau:  0.1 , score:  4317.859459670564
bw:  1.2 , tau:  0.1 , score:  4317.859459670564
bw:  1.2 , tau:  0.1 , score:  4317.859459670564
bw:  1.0 , tau:  0.1 , score:  4273.10013420979
bw:  0.9 , tau:  0.1 , score:  4269.090130007157
bw:  0.9 , tau:  0.1 , score:  4269.090130007157
bw:  0.9 , tau:  0.1 , score:  4269.090130007157
time cost: 0:03:4.091
Spatiotemporal Weighted Regression (GTWR) Results
---------------------------------------------------------------------------
Spatial kernel:                                          Fixed gaussian
Model tau used:                                                       0.100
Spatial Bandwidth used:                                               0.900
Temporal Bandwidth used:                                              2.846

Diagnostic information
--------------------------------------

../GTWR_ana\regre_gtwr.py:173: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(result_data_save_file)


In [26]:
## KSI
regre_gtwr.regression.gtwr_start(data_file="../../data/merge_data/inj_data_norm/inj_merge_data.shp",
                          explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18'],
                          result_data_save_file="../../data/GTWR_result/inj_betas/betas.shp")

bw:  8.3 , tau:  2.7 , score:  5995.095067020855
bw:  10.2 , tau:  4.3 , score:  5994.928166752898
bw:  11.5 , tau:  5.5 , score:  5994.866533074843
bw:  12.2 , tau:  6.3 , score:  5994.843735904936
bw:  12.7 , tau:  6.8 , score:  5994.830526976815
bw:  13.0 , tau:  7.2 , score:  5994.823641051885
bw:  13.1 , tau:  7.3 , score:  5994.821396243698
bw:  13.3 , tau:  7.5 , score:  5994.817392647279
bw:  13.3 , tau:  7.5 , score:  5994.817392647279
time cost: 0:02:23.782
Spatiotemporal Weighted Regression (GTWR) Results
---------------------------------------------------------------------------
Spatial kernel:                                          Fixed gaussian
Model tau used:                                                       7.500
Spatial Bandwidth used:                                              13.300
Temporal Bandwidth used:                                              4.856

Diagnostic information
---------------------------------------------------------------------------
Re

../GTWR_ana\regre_gtwr.py:173: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(result_data_save_file)


## 2.3 GWR

In [27]:
## PDO
regre_gtwr.regression.gwr_start(data_file="../../data/merge_data/pdo_data_norm/pdo_merge_data.shp",
                         explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18'])

bw: 15.0 , score: 5130.26
bw: 10.0 , score: 5119.11
bw: 6.0 , score: 5081.5
bw: 4.0 , score: 5005.52
bw: 2.0 , score: 4753.84
bw: 2.0 , score: 4753.84
time cost: 0:00:8.487
Geographically weighted regression (GWR) Results
---------------------------------------------------------------------------
Spatial kernel:                                          Fixed gaussian
Temporal Bandwidth used:                                              2.000

Diagnostic information
---------------------------------------------------------------------------
Residual sum of squares:                                           1071.791
Effective number of parameters (trace(S)):                           52.932
Degree of freedom (n - trace(S)):                                  2131.068
Sigma estimate:                                                       0.709
Log-likelihood:                                                   -2321.647
AIC:                                                               4751.15

In [28]:
## KSI
regre_gtwr.regression.gwr_start(data_file="../../data/merge_data/inj_data_norm/inj_merge_data.shp",
                         explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18'])

bw: 15.0 , score: 6007.8
bw: 10.0 , score: 6006.88
bw: 6.0 , score: 6004.18
bw: 4.0 , score: 5999.43
bw: 2.0 , score: 5985.73
bw: 2.0 , score: 5985.73
time cost: 0:00:8.59
Geographically weighted regression (GWR) Results
---------------------------------------------------------------------------
Spatial kernel:                                          Fixed gaussian
Temporal Bandwidth used:                                              2.000

Diagnostic information
---------------------------------------------------------------------------
Residual sum of squares:                                           1884.300
Effective number of parameters (trace(S)):                           52.751
Degree of freedom (n - trace(S)):                                  2131.249
Sigma estimate:                                                       0.940
Log-likelihood:                                                   -2937.781
AIC:                                                               5983.064

## 2.4 OLS

In [29]:
## PDO
regre_gtwr.regression.ols_start(data_file="../../data/merge_data/pdo_data_norm/pdo_merge_data.shp",
                         explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18'])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.389
Method:                 Least Squares   F-statistic:                     107.9
Date:                Fri, 17 Feb 2023   Prob (F-statistic):          1.07e-223
Time:                        16:42:16   Log-Likelihood:                -2554.4
No. Observations:                2184   AIC:                             5137.
Df Residuals:                    2170   BIC:                             5216.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.776e-17      0.017   1.66e-15      1.0

In [30]:
## KSI
regre_gtwr.regression.ols_start(data_file="../../data/merge_data/inj_data_norm/inj_merge_data.shp",
                         explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18'])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     17.65
Date:                Fri, 17 Feb 2023   Prob (F-statistic):           1.95e-39
Time:                        16:42:16   Log-Likelihood:                -2989.2
No. Observations:                2184   AIC:                             6006.
Df Residuals:                    2170   BIC:                             6086.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.734e-16      0.020   2.32e-14      1.0

## 3.1 Analysis of the Characteristics of Time Dimension

In [1]:
## PDO
explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18']
for i in range(len(explain_var_list)):
    tmp_col = explain_var_list[i]+"_betas"
    print("正在可视化{0}".format(tmp_col))
    tmp_save_file = "../../data/GTWR_result/pdo_result/" + tmp_col + "_vis.csv"
    data_st_vis.vis.temporal_vis(beta_file="../../data/GTWR_result/pdo_betas/betas.shp",
                         column_name=tmp_col,
                         data_save_file=tmp_save_file)

正在可视化x1_betas


NameError: name 'data_st_vis' is not defined

In [2]:
## KSI
explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18']
for i in range(len(explain_var_list)):
    tmp_col = explain_var_list[i]+"_betas"
    print("正在可视化{0}".format(tmp_col))
    tmp_save_file = "../../data/GTWR_result/inj_result/" + tmp_col + "_vis.csv"
    data_st_vis.vis.temporal_vis(beta_file="../../data/GTWR_result/inj_betas/betas.shp",
                         column_name=tmp_col,
                         data_save_file=tmp_save_file)

正在可视化x1_betas


NameError: name 'data_st_vis' is not defined

## 3.2 Analysis of the Characteristics of Spatial Dimensions

In [3]:
## PDO
explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18']
for i in range(len(explain_var_list)):
    tmp_col = explain_var_list[i]+"_betas"
    print("正在可视化{0}".format(tmp_col))
    tmp_save_path = "../../data/GTWR_result/pdo_result/" + tmp_col + "_vis/"
    os.mkdir(tmp_save_path)
    data_st_vis.vis.spatial_vis(beta_file="../../data/GTWR_result/pdo_betas/betas.shp",
                    column_name=tmp_col,
                    dev_file="../../data/distinct_camera.xlsx",
                    data_save_file=tmp_save_path + tmp_col + "_vis.shp",
                    image_save_path="../../data/GTWR_result/tile_file/")

正在可视化x1_betas


NameError: name 'os' is not defined

In [4]:
## KSI
explain_var_list=['x1','x4','x5','x7','x8','x10','x12','x13','x14','x15','x16','x17','x18']
for i in range(len(explain_var_list)):
    tmp_col = explain_var_list[i]+"_betas"
    print("正在可视化{0}".format(tmp_col))
    tmp_save_path = "../../data/GTWR_result/inj_result/" + tmp_col + "_vis/"
    os.mkdir(tmp_save_path)
    data_st_vis.vis.spatial_vis(beta_file="../../data/GTWR_result/inj_betas/betas.shp",
                    column_name=tmp_col,
                    dev_file="../../data/distinct_camera.xlsx",
                    data_save_file=tmp_save_path + tmp_col + "_vis.shp",
                    image_save_path="../../data/GTWR_result/tile_file/")

正在可视化x1_betas


NameError: name 'os' is not defined